In [1]:
import numpy as np
import json
from bs4 import BeautifulSoup
import plotly.graph_objects as go


METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}

ENCOUNTER_DIFFICULTIES = {
    'Easy':   {
        'XP': [25,50,75,125,250,300,350,450,550,600,800,1000,1100,1250,1400,1600,2000,2100,2400,2800], 
        'color': '#1F77B4'
    },
    'Medium': {
        'XP': [50,100,150,250,500,600,750,900,1100,1200,1600,2000,2200,2500,2800,3200,3900,4200,4900,5700], 
        'color': '#FF7F0E'
    },
    'Hard':   {
        'XP': [75,150,225,375,750,900,1100,1400,1600,1900,2400,3000,3400,3800,4300,4800,5900,6300,7300,8500], 
        'color': '#2CA02C'
    },
    'Deadly': {
        'XP': [100,200,400,500,1100,1400,1700,2100,2400,2800,3600,4500,5100,5700,6400,7200,8800,9500,10900,12700], 
        'color': '#D62728'
    },
    'Daily':  {
        'XP': [300,600,1200,1700,3500,4000,5000,6000,7500,9000,10500,11500,13500,15000,18000,20000,25000,27000,30000,40000], 
        'color': '#9467BD'
    },
}

In [2]:
# Setup template and html config for plotly figures.
TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=True, # must be True to auto-scale when resizing
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        legend=dict(
            #bordercolor='Black',
            #borderwidth=1,
            font_size=10,
            #tracegroupgap=5,
            #orientation='h',
        )
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': True, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}


In [3]:
# Plots average daily XP for PCs, calculated for a full adventuring day with two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated daily XP / Daily XP',
            range=[0, 1.6],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='bottom',  y=0.01,
            xanchor='right', x=0.99,
            orientation='v',
        )
    )
)

enc_XP_budget = np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
for diff in ['Easy','Medium','Hard','Deadly']:
    d = ENCOUNTER_DIFFICULTIES[diff]
    pc_data_file = f'../xp-and-player-characters/pc data - {diff} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)

    eXP = np.zeros(20)
    for pc_class in PC_CLASSES:
        eXP += np.array([x['XP mean'] for x in pc_data[pc_class]])
    eXP /= len(PC_CLASSES)

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eXP/enc_XP_budget,
        mode='lines+markers', 
        line=dict(color=d['color'], dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        name=diff + ' encounters',
        hovertemplate=
            '<b>' + diff + ' encounters</b><br>'+
            'lvl %{x}<br>'+
            'XP ratio %{y:.2f}'+
            '<extra></extra>'
    ))

ref_XP = np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
fig.add_trace(go.Scatter(
    x=levels, 
    y=ref_XP/enc_XP_budget,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    name='Daily XP budget',
    showlegend=False,
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-full-daily-xp-vs-level-adventuring-days-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-full-daily-xp-vs-level-adventuring-days-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [4]:
# Plots average daily effective DPR for PCs, calculated for a full adventuring day with two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='effective damage per round',
            range=[0, 100],
            tick0=0, dtick=10.0,
            minor=dict(tick0=0, dtick=5.0),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            orientation='v',
        )
    )
)

for diff in ['Easy','Medium','Hard','Deadly']:
    d = ENCOUNTER_DIFFICULTIES[diff]
    pc_data_file = f'../xp-and-player-characters/pc data - {diff} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)

    eDPR = np.zeros(20)
    for pc_class in PC_CLASSES:
        eDPR += np.array([x['effective damage per round mean'] for x in pc_data[pc_class]])
    eDPR /= len(PC_CLASSES)

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eDPR,
        mode='lines+markers', 
        line=dict(color=d['color'], dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        name=diff + ' encounters',
        hovertemplate=
            '<b>' + diff + ' encounters</b><br>'+
            'lvl %{x}<br>'+
            'eDPR %{y:.1f}'+
            '<extra></extra>'
    ))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-full-daily-edpr-vs-level-adventuring-days-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-full-daily-edpr-vs-level-adventuring-days-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [5]:
# Plots average daily XP for PCs relative to Medium adventuring day, calculated for a full adventuring day with two short rests.
levels = np.array(range(1,21,1))

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='daily XP / Medium daily XP',
            range=[0.8, 1.3],
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.05),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            orientation='h',
        )
    )
)

enc_XP_budget = np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])
diffs = {
    'Easy': {
        'daily XP': np.zeros(20)
    },
    'Medium': {
        'daily XP': np.zeros(20)
    },
    'Hard': {
        'daily XP': np.zeros(20)
    },
    'Deadly': {
        'daily XP': np.zeros(20)
    }
}
for d in diffs:
    diff = diffs[d]
    ediff = ENCOUNTER_DIFFICULTIES[d]
    pc_data_file = f'../xp-and-player-characters/pc data - {d} 2 short rests.json'

    with open(pc_data_file, 'r') as fin:
        pc_data = json.load(fin)

    enc_XP = np.zeros(20)
    for pc_class in PC_CLASSES:
        enc_XP += np.array([x['XP mean'] for x in pc_data[pc_class]])
    enc_XP /= len(PC_CLASSES)
    diff['daily XP'] = enc_XP/enc_XP_budget

for d in diffs:
    y = diffs[d]['daily XP']/diffs['Medium']['daily XP']
    fig.add_trace(go.Scatter(
        x=levels, 
        y=y,
        mode='markers+lines', 
        #line=dict(color='black', dash='dash'),
        name=d+' encounters',
        showlegend=True,
        hovertemplate=
            '<b>' + d + ' encounters</b><br>'+
            'lvl %{x}<br>'+
            'XP ratio %{y:.2f}'+
            '<extra></extra>'
    ))
    print(d, y.mean(), np.median(y))

# show figure
fig.show(config=TFB_CONFIG)

# save small format figure
if SAVEFIGS:
    file_name = f'./fig-daily-xp-ratio-vs-level-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-daily-xp-ratio-vs-level-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

Easy 0.9171542987995448 0.9116259477095588
Medium 1.0 1.0
Hard 1.055394325976669 1.0564325231623914
Deadly 1.1162567096155165 1.1201164734924733


In [7]:
# plots the ratio between the Deadly and Medium daily XPs for each classes.
from bs4 import BeautifulSoup

enc_diffs = ['Easy','Medium','Hard','Deadly']
levels = np.array(range(1,21,1))

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[-0.2, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='Deadly / Medium encounter XP',
            range=[0.90, 1.5],
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.05),
        ),
        legend=dict(
            yanchor='bottom',  y=1.04,
            xanchor='left', x=0.01,
            orientation='h',
        )
    )
)

groups = {
    'martial': {
        'classes': [], 
        'xp mean': np.zeros(len(levels))
    },
    'martial-spellcaster': {
        'classes': [], 
        'xp mean': np.zeros(len(levels))
    },
    'spellcaster': {
        'classes': [], 
        'xp mean': np.zeros(len(levels))
    },
    'average': {
        'classes': [], 
        'xp mean': np.zeros(len(levels))
    }
}
xp_mean = np.zeros(len(levels))
for pc in PC_CLASSES:
    pc_class = PC_CLASSES[pc]
    pc_xp = np.zeros([20, len(enc_diffs)])
    for i in range(len(enc_diffs)):
        diff = enc_diffs[i]
        with open(f'../xp-and-player-characters/pc data - {diff} 2 short rests.json', 'r') as fin:
            pc_data = json.load(fin)
        
        #pc_xp[:,i] = np.array(pc_data[pc]['daily XP mean'])
        pc_xp[:,i] = np.array([x['XP mean'] for x in pc_data[pc]])

    for i in range(20):
        pc_xp[i,:] /= pc_xp[i,1]

    fig.add_trace(go.Scatter(
        x=levels, 
        y=pc_xp[:,3],
        mode='lines+markers', 
        line=dict(color=pc_class['color'], dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'level %{x}<br>'+
            'XP delta %{y:.2f}'+
            '<extra></extra>'
    ))
    
    groups[pc_class['group']]['classes'] += [pc]
    groups[pc_class['group']]['xp mean'] += pc_xp[:,3]
    groups['average']['classes'] += [pc]
    groups['average']['xp mean'] += pc_xp[:,3]

for g in ['average']:
    group = groups[g]

    group['xp mean'] /= len(group['classes'])

    fig.add_trace(go.Scatter(
        x=levels, 
        y=group['xp mean'],
        mode='lines', 
        line=dict(color='black', dash='dash'),
        unselected=dict(marker=dict(opacity=0.0)),
        name=g,
        hovertemplate=
            '<b>'+g+'</b><br>'+
            'level %{x}<br>'+
            'XP delta %{y:.2f}'+
            '<extra></extra>'
    ))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-class-daily-xp-ratio-vs-level-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    fig_soup.div['style'] = 'aspect-ratio:600/550'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10), legend=dict(font_size=8))
    file_name = f'./fig-class-daily-xp-ratio-vs-level-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    fig_soup.div['style'] = 'aspect-ratio:600/600'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))